In [1]:
# -*- coding: utf-8 -*-

import pandas as pd
import numpy as np
from datetime import datetime

def input():

    # file input
    data = pd.read_csv('trade.csv', usecols=['uid', 'vipno', 'sldat', 'pluno'])
    data['timestamp'] = pd.to_datetime(data['sldat'])

    # sort 
    data.sort_values(['vipno','timestamp'],ascending=[1,1],inplace=True) 

    # make groups 
    data['rank'] = data['timestamp'].groupby(data['vipno']).rank(ascending=0,method='first')

    # take top 60% in every group
    grouped = data.groupby(['vipno'], as_index = True).apply(lambda x: x[x['rank'] <= (0.6 * x['rank'].max())])

    # convert
    data_set = grouped.drop(['rank', 'timestamp', 'sldat', 'vipno'], axis=1).reset_index(drop=True)

    # merge by uid
    data_set['value'] = data_set['pluno']
    data_set = data_set.pivot_table(data_set, index=['uid'], columns=['pluno'])

    return data_set.fillna(0)

def input_new():

    # file input
    data = pd.read_csv('trade_new.csv', usecols=['uid', 'vipno', 'sldatime', 'pluno'])
    data['timestamp'] = pd.to_datetime(data['sldatime'])

    # sort 
    data.sort_values(['vipno','timestamp'],ascending=[1,1],inplace=True) 

    # make groups 
    data['rank'] = data['timestamp'].groupby(data['vipno']).rank(ascending=0,method='first')

    # take top 60% in every group
    grouped = data.groupby(['vipno'], as_index = True).apply(lambda x: x[x['rank'] <= (0.6 * x['rank'].max())])

    # convert
    data_set = grouped.drop(['rank', 'timestamp', 'sldatime', 'vipno'], axis=1).reset_index(drop=True)

    # merge by uid
    data_set['value'] = data_set['pluno']
    data_set = data_set.pivot_table(data_set, index=['uid'], columns=['pluno'])

    return data_set.fillna(0)

def createInitSet(data_set):  

    # clear all 0
    data_array = data_set.as_matrix()
    data_dok = []
    for row in data_array:
        data_dok.append([x for x in row if x != 0.0])

    # transform array to dict
    retDict = {}  
    for trans in data_dok:  
        retDict[frozenset(trans)] = 1  

    return retDict

In [2]:
class treeNode:

    def __init__(self, name_value, num_occur, parent_node):

        self.name = name_value  
        self.count = num_occur  
        self.node_link = None  
        self.parent = parent_node  
        self.children = {}  

    def increase(self, num_occur):

        self.count += num_occur

    def disp(self, ind=1):

        print '  ' * ind, self.name, ' ', self.count
        for child in self.children.values():
            child.disp(ind + 1)

def create_tree(data_set, min_support=1):

    """
    创建FP树
    :param data_set: 数据集
    :param min_support: 最小支持度
    :return:
    """

    freq_items = {}  # 频繁项集
    for trans in data_set:  # 第一次遍历数据集
        for item in trans:
            freq_items[item] = freq_items.get(item, 0) + data_set[trans]

    # print freq_items
    header_table = {k: v for (k, v) in freq_items.iteritems() if v >= min_support}  # 创建头指针表
    # for key in header_table:
    #     print key, header_table[key]

    # 无频繁项集
    if len(header_table) == 0:
        return None, None
    for k in header_table:
        header_table[k] = [header_table[k], None]  # 添加头指针表指向树中的数据

    # 创建树过程
    ret_tree = treeNode('Null Set', 1, None)  # 根节点

    # 第二次遍历数据集
    for trans, count in data_set.items():
        local_data = {}
        for item in trans:
            if header_table.get(item, 0):
                local_data[item] = header_table[item][0]
        if len(local_data) > 0:
            ##############################################################################################
            # 这里修改机器学习实战中的排序代码：
            ordered_items = [v[0] for v in sorted(local_data.items(), key=lambda kv: (-kv[1], kv[0]))]
            ##############################################################################################
            update_tree(ordered_items, ret_tree, header_table, count)  # populate tree with ordered freq itemset
            
    return ret_tree, header_table


def update_tree(items, in_tree, header_table, count):

    '''
    :param items: 元素项
    :param in_tree: 检查当前节点
    :param header_table:
    :param count:
    :return:
    '''
    if items[0] in in_tree.children:  # check if ordered_items[0] in ret_tree.children
        in_tree.children[items[0]].increase(count)  # incrament count
    else:  # add items[0] to in_tree.children
        in_tree.children[items[0]] = treeNode(items[0], count, in_tree)
        if header_table[items[0]][1] is None:  # update header table
            header_table[items[0]][1] = in_tree.children[items[0]]
        else:
            update_header(header_table[items[0]][1], in_tree.children[items[0]])

    if len(items) > 1:  # call update_tree() with remaining ordered items
        update_tree(items[1::], in_tree.children[items[0]], header_table, count)


def update_header(node_test, target_node):

    '''
    :param node_test:
    :param target_node:
    :return:
    '''
    while node_test.node_link is not None:  # Do not use recursion to traverse a linked list!
        node_test = node_test.node_link

    node_test.node_link = target_node


def ascend_tree(leaf_node, pre_fix_path):

    '''
    遍历父节点，找到路径
    :param leaf_node:
    :param pre_fix_path:
    :return:
    '''
    if leaf_node.parent is not None:
        pre_fix_path.append(leaf_node.name)
        ascend_tree(leaf_node.parent, pre_fix_path)


def find_pre_fix_path(base_pat, tree_node):

    '''
    创建前缀路径
    :param base_pat: 频繁项
    :param treeNode: FP树中对应的第一个节点
    :return:
    '''
    # 条件模式基
    cond_pats = {}

    while tree_node is not None:
        pre_fix_path = []
        ascend_tree(tree_node, pre_fix_path)
        if len(pre_fix_path) > 1:
            cond_pats[frozenset(pre_fix_path[1:])] = tree_node.count
        tree_node = tree_node.node_link

    return cond_pats


def mine_tree(in_tree, header_table, min_support, pre_fix, freq_items):

    '''
    挖掘频繁项集
    :param in_tree:
    :param header_table:
    :param min_support:
    :param pre_fix:
    :param freq_items:
    :return:
    '''
    # print header_table
    # 从小到大排列table中的元素，为遍历寻找频繁集合使用
    bigL = [v[0] for v in sorted(header_table.items(), key=lambda p: p[1])]  # (sort header table)

    for base_pat in bigL:  # start from bottom of header table

        new_freq_set = pre_fix.copy()
        new_freq_set.add(base_pat)

        # print 'finalFrequent Item: ',new_freq_set    #append to set
        if len(new_freq_set) > 0:
            freq_items[frozenset(new_freq_set)] = header_table[base_pat][0]

        cond_patt_bases = find_pre_fix_path(base_pat, header_table[base_pat][1])
        my_cond_tree, my_head = create_tree(cond_patt_bases, min_support)

        # print 'head from conditional tree: ', my_head
        if my_head is not None:  # 3. mine cond. FP-tree
            # print 'conditional tree for: ',new_freq_set
            # my_cond_tree.disp(1)
            mine_tree(my_cond_tree, my_head, min_support, new_freq_set, freq_items)


def fp_growth(data_set, min_support=1):

    my_fp_tree, my_header_tab = create_tree(data_set, min_support)
    # my_fp_tree.disp()
    freq_items = {}
    mine_tree(my_fp_tree, my_header_tab, min_support, set([]), freq_items)

    return freq_items


In [3]:
if __name__ == '__main__':

    data_set = input()

    # min_support = [2,4,8,16,32,64]
    for min_support in [2,4,8,16,32,64]:
        print "\nmin_support = %d: "%min_support
        frequent_sets = fp_growth(createInitSet(data_set), min_support).items()
        for k, v in frequent_sets:
            print "pattern:", k, "support:", v

    data_set = input_new()

    # min_support = [2,4,8,16,32,64]
    for min_support in [2,4,8,16,32,64]:
        print "\nmin_support = %d: "%min_support
        frequent_sets = fp_growth(createInitSet(data_set), min_support).items()
        for k, v in frequent_sets:
            print "pattern:", k, "support:", v



min_support = 2: 
pattern: frozenset([14222034.0]) support: 7
pattern: frozenset([10131021.0]) support: 2
pattern: frozenset([25111048.0, 25111021.0]) support: 3
pattern: frozenset([15231004.0, 27400855.0]) support: 2
pattern: frozenset([23110009.0, 30380002.0, 22671077.0, 15200007.0]) support: 2
pattern: frozenset([25120016.0, 30380003.0, 27000581.0]) support: 3
pattern: frozenset([27410000.0, 27002555.0, 27000581.0, 27000582.0]) support: 2
pattern: frozenset([24401008.0, 23131002.0, 15202079.0]) support: 2
pattern: frozenset([22100010.0, 27300274.0]) support: 5
pattern: frozenset([22111004.0]) support: 27
pattern: frozenset([15114003.0, 22500022.0]) support: 2
pattern: frozenset([27410000.0, 30380003.0]) support: 8
pattern: frozenset([30323001.0]) support: 4
pattern: frozenset([25120016.0, 22103001.0, 23132068.0]) support: 2
pattern: frozenset([22702005.0]) support: 2
pattern: frozenset([22111024.0]) support: 2
pattern: frozenset([27400118.0]) support: 2
pattern: frozenset([14916008

pattern: frozenset([22036000.0, 30380002.0]) support: 4
pattern: frozenset([15130035.0]) support: 5
pattern: frozenset([10110017.0]) support: 4
pattern: frozenset([10131020.0]) support: 3
pattern: frozenset([15120000.0, 15114037.0]) support: 2
pattern: frozenset([30380002.0, 24101006.0]) support: 4
pattern: frozenset([27002555.0, 22102014.0]) support: 2
pattern: frozenset([27410000.0, 25130010.0, 27000581.0, 27000582.0]) support: 2
pattern: frozenset([27410000.0, 25101044.0, 27000581.0, 27000582.0, 27002555.0, 22103005.0]) support: 2
pattern: frozenset([15605006.0]) support: 2
pattern: frozenset([22701014.0]) support: 6
pattern: frozenset([27000573.0, 27100542.0]) support: 2
pattern: frozenset([14515001.0, 27300274.0, 22102005.0, 15114013.0]) support: 2
pattern: frozenset([10132002.0]) support: 2
pattern: frozenset([27410000.0, 22000049.0, 25101044.0, 27000581.0, 27000582.0, 22030005.0]) support: 2
pattern: frozenset([23110193.0, 27300273.0]) support: 2
pattern: frozenset([30380003.0, 

pattern: frozenset([11110004.0]) support: 3
pattern: frozenset([15115008.0]) support: 7
pattern: frozenset([27002560.0, 23110009.0]) support: 2
pattern: frozenset([15120000.0, 15114008.0]) support: 3
pattern: frozenset([22103010.0, 24101003.0]) support: 2
pattern: frozenset([27300274.0, 23132068.0, 14082015.0]) support: 2
pattern: frozenset([15200001.0, 30380003.0, 15114005.0]) support: 2
pattern: frozenset([10300043.0, 15113004.0, 22008019.0]) support: 2
pattern: frozenset([15004041.0]) support: 3
pattern: frozenset([10141019.0]) support: 3
pattern: frozenset([27410000.0, 23134003.0, 25101044.0]) support: 2
pattern: frozenset([27300274.0, 22130007.0]) support: 2
pattern: frozenset([15115043.0]) support: 3
pattern: frozenset([30380002.0, 15200007.0, 22671077.0, 15114015.0]) support: 2
pattern: frozenset([27000033.0, 22102014.0]) support: 2
pattern: frozenset([22000049.0, 23134003.0]) support: 2
pattern: frozenset([22030005.0, 27000582.0, 27000581.0]) support: 3
pattern: frozenset([2741

pattern: frozenset([22036000.0, 27000582.0]) support: 2
pattern: frozenset([10000011.0, 27200924.0]) support: 2
pattern: frozenset([15116003.0]) support: 2
pattern: frozenset([10422010.0, 30380002.0]) support: 2
pattern: frozenset([22103011.0, 15200004.0, 22103005.0]) support: 2
pattern: frozenset([27410000.0, 27300281.0, 30380002.0]) support: 2
pattern: frozenset([22102000.0, 30380002.0]) support: 2
pattern: frozenset([14721022.0]) support: 3
pattern: frozenset([25120016.0, 27000582.0]) support: 4
pattern: frozenset([23142001.0, 23110009.0]) support: 3
pattern: frozenset([27240000.0, 23110009.0]) support: 2
pattern: frozenset([22100000.0, 22111004.0, 22102005.0]) support: 3
pattern: frozenset([30380003.0, 27000574.0]) support: 3
pattern: frozenset([15114032.0]) support: 4
pattern: frozenset([22103003.0]) support: 4
pattern: frozenset([10151000.0]) support: 3
pattern: frozenset([27002173.0, 15202079.0]) support: 2
pattern: frozenset([23120001.0, 14013010.0, 30380003.0]) support: 2
patt

pattern: frozenset([14015065.0, 27300274.0]) support: 2
pattern: frozenset([24011018.0]) support: 4
pattern: frozenset([14222033.0, 14222034.0]) support: 2
pattern: frozenset([22111004.0, 24101006.0]) support: 3
pattern: frozenset([27300281.0]) support: 7
pattern: frozenset([25101002.0, 23132068.0]) support: 2
pattern: frozenset([22100010.0, 22102014.0]) support: 4
pattern: frozenset([10133001.0]) support: 7
pattern: frozenset([14000002.0, 25101044.0, 27300274.0]) support: 2
pattern: frozenset([22601000.0, 27410000.0]) support: 2
pattern: frozenset([23110009.0, 23113019.0]) support: 2
pattern: frozenset([15242044.0]) support: 5
pattern: frozenset([10137009.0, 10422010.0, 30380002.0, 15004053.0]) support: 2
pattern: frozenset([25111048.0]) support: 57
pattern: frozenset([15110036.0, 15110007.0]) support: 2
pattern: frozenset([15004024.0]) support: 5
pattern: frozenset([10150032.0, 27410003.0, 10000011.0]) support: 2
pattern: frozenset([10119057.0]) support: 3
pattern: frozenset([2210000

...

pattern: frozenset([32917000.0, 27410000.0]) support: 2
pattern: frozenset([25101044.0, 27000581.0]) support: 4
pattern: frozenset([23134003.0, 23110006.0]) support: 2
pattern: frozenset([22036000.0, 22103006.0]) support: 2
pattern: frozenset([23134003.0]) support: 44
pattern: frozenset([30380003.0, 14802054.0]) support: 2
pattern: frozenset([22111004.0, 22500022.0]) support: 3
pattern: frozenset([15202039.0]) support: 5
pattern: frozenset([22603000.0, 22601000.0]) support: 2
pattern: frozenset([22603000.0, 22603001.0]) support: 2
pattern: frozenset([11220014.0]) support: 2
pattern: frozenset([14815012.0]) support: 2
pattern: frozenset([30380001.0, 15200012.0]) support: 2
pattern: frozenset([15110010.0]) support: 4
pattern: frozenset([25120016.0, 30380002.0, 22102013.0, 25120014.0]) support: 2
pattern: frozenset([25405948.0, 27000581.0]) support: 2
pattern: frozenset([30351007.0]) support: 3
pattern: frozenset([22111004.0, 30380003.0, 22102004.0, 22102005.0]) support: 2
pattern: frozen

pattern: frozenset([15130009.0, 30380003.0]) support: 7
pattern: frozenset([30380003.0, 15115028.0]) support: 5
pattern: frozenset([22034004.0]) support: 6
pattern: frozenset([15115002.0, 30380002.0]) support: 6
pattern: frozenset([25150001.0]) support: 4
pattern: frozenset([15202044.0]) support: 20
pattern: frozenset([15116001.0]) support: 8
pattern: frozenset([10136008.0]) support: 4
pattern: frozenset([22101001.0, 30380003.0]) support: 7
pattern: frozenset([15440002.0, 30380003.0]) support: 4
pattern: frozenset([15110032.0]) support: 4
pattern: frozenset([30380003.0, 22102005.0]) support: 15
pattern: frozenset([14802057.0]) support: 6
pattern: frozenset([22102013.0]) support: 14
pattern: frozenset([22034000.0, 30380002.0]) support: 6
pattern: frozenset([15115003.0, 30380003.0]) support: 5
pattern: frozenset([10001007.0]) support: 10
pattern: frozenset([22111004.0]) support: 41
pattern: frozenset([10132017.0]) support: 6
pattern: frozenset([27410000.0, 30380003.0]) support: 16
patter

pattern: frozenset([14721021.0]) support: 5
pattern: frozenset([22000049.0]) support: 16
pattern: frozenset([14721022.0]) support: 5
pattern: frozenset([14130055.0]) support: 12
pattern: frozenset([14860020.0]) support: 12
pattern: frozenset([22036000.0, 22102005.0]) support: 9
pattern: frozenset([24101002.0]) support: 19
pattern: frozenset([25100113.0]) support: 4
pattern: frozenset([22170001.0, 30380003.0]) support: 7
pattern: frozenset([15230001.0]) support: 18
pattern: frozenset([24101006.0, 27000574.0]) support: 4
pattern: frozenset([27002169.0]) support: 6
pattern: frozenset([30380003.0, 22610103.0]) support: 5
pattern: frozenset([22830005.0]) support: 4
pattern: frozenset([15114032.0]) support: 5
pattern: frozenset([27002177.0]) support: 4
pattern: frozenset([14300086.0]) support: 5
pattern: frozenset([22170001.0, 22102014.0]) support: 6
pattern: frozenset([14900009.0]) support: 4
pattern: frozenset([10152013.0]) support: 10
pattern: frozenset([14900010.0]) support: 4
pattern: f

pattern: frozenset([22171000.0]) support: 7
pattern: frozenset([23110192.0]) support: 4
pattern: frozenset([10110017.0]) support: 10
pattern: frozenset([25120016.0, 22102014.0]) support: 6
pattern: frozenset([27410000.0, 27300273.0]) support: 10
pattern: frozenset([27400708.0, 27400126.0]) support: 4
pattern: frozenset([30380002.0, 15231005.0]) support: 4
pattern: frozenset([15130027.0]) support: 51
pattern: frozenset([15263006.0]) support: 4
pattern: frozenset([30380002.0, 24101006.0]) support: 5
pattern: frozenset([14916005.0]) support: 7
pattern: frozenset([22132010.0, 30380002.0]) support: 4
pattern: frozenset([27300273.0, 23110009.0]) support: 4
pattern: frozenset([22036000.0, 15110001.0]) support: 6
pattern: frozenset([27300284.0]) support: 4
pattern: frozenset([24101000.0]) support: 13
pattern: frozenset([22005006.0]) support: 4
pattern: frozenset([11056085.0]) support: 8
pattern: frozenset([10151002.0]) support: 11
pattern: frozenset([25101011.0]) support: 24
pattern: frozenset

pattern: frozenset([27410000.0]) support: 96
pattern: frozenset([15230001.0]) support: 18
pattern: frozenset([15202044.0]) support: 20
pattern: frozenset([15119001.0]) support: 37
pattern: frozenset([27002555.0]) support: 17
pattern: frozenset([22102014.0]) support: 96
pattern: frozenset([25101044.0]) support: 65
pattern: frozenset([24101002.0]) support: 19
pattern: frozenset([22111004.0]) support: 41
pattern: frozenset([27410000.0, 30380003.0]) support: 16
pattern: frozenset([22102005.0, 22102014.0]) support: 18
pattern: frozenset([15231005.0]) support: 17
pattern: frozenset([21801125.0]) support: 17
pattern: frozenset([22100000.0]) support: 21
pattern: frozenset([22008021.0]) support: 30
pattern: frozenset([22034000.0]) support: 16
pattern: frozenset([30380001.0]) support: 28
pattern: frozenset([23110009.0, 30380003.0]) support: 33
pattern: frozenset([27300276.0]) support: 16
pattern: frozenset([22002240.0]) support: 27
pattern: frozenset([15130027.0]) support: 51
pattern: frozenset(